# Лабораторная работа № 4

## Реализация градиентного спуска

Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью:

**Задание 1** Градиентного спуска;

**Задание 2** Стохастического градиентного спуска.


Во всех пунктах необходимо соблюдать следующие условия:

* Все вычисления должны быть векторизованы;
* Циклы средствами python допускается использовать только для итераций градиентного спуска;
* В качестве критерия останова необходимо использовать (одновременно):

    * проверку на евклидовую норму разности весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, задаваемого параметром `tolerance`);
    * достижение максимального числа итераций (например, 10000, задаваемого параметром `max_iter`).
* Чтобы проследить, что оптимизационный процесс действительно сходится, будем использовать атрибут класса `loss_history` — в нём после вызова метода `fit` должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту);
* Инициализировать веса можно случайным образом или нулевым вектором. 


Ниже приведён шаблон класса, который должен содержать код реализации каждого из методов.

## Формула градиента

Функция потерь в векторном (матричном) виде можно представить следующим образом:

$$Q(w) = \frac{ 1 }{ n } \|X w - y\|^2$$


Тогда производная этой функции будет принимать следующий вид (производная вычисляется как **производная сложной функции**):

$$\frac{ dQ }{ d w }= \frac{ 1 }{ n } (\|X w - y\|^2)' * (X w - y)'  =  \frac{ 1 }{ n } 2(X w -y)X = \frac{ 1 }{ n } 2X(Xw-y)$$

Но, т. к. здесь используются матрицы, то $X$ перед скобками необходимо транспонировать, чтобы было возможным осуществить операцию векторного умножения, нужно соблюсти правило:
> Чтобы матрицу A можно было умножить на матрицу B нужно, чтобы число столбцов матрицы A равнялось числу строк матрицы B.

Поэтому, производная функции потерь будет иметь следующий вид:

$$\frac{ dQ }{ d w }= \frac{ 1 }{ n } 2X^{T}(Xw-y)$$

Т. к. в выражении 1 переменная, то градиент будет выглядеть так:

$$\nabla Q(w) = \frac{ 1 }{ n } 2X^{T}(Xw-y) $$

Подробнее см. раздел 5.3  Линейная регрессия в [Математические методы обучения по прецедентам (теория обучения машин)](http://www.machinelearning.ru/wiki/images/6/6d/Voron-ML-1.pdf)

In [17]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator

class LinearReg(BaseEstimator):
    def __init__(self, gd_type='stochastic', 
                 tolerance=1e-4, max_iter=1000, w0=None, alpha=0, eta=1e-5):
        """
        gd_type: 'full' or 'stochastic'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        eta: learning rate
        """
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None 
    
    def fit(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: self
        """
        self.loss_history = []
        
        n_samples, n_features = X.shape
        
        if self.w0 is not None:
            self.w = self.w0.copy()
        else:
            self.w = np.random.normal(0, 0.01, n_features + 1)
        
        X_with_bias = np.c_[np.ones(n_samples), X]
        w_prev = self.w.copy()
        
        for iteration in range(self.max_iter):
            if self.gd_type == 'full':
                gradient = self.calc_gradient(X_with_bias, y)
                self.w = self.w - self.eta * gradient
                
            elif self.gd_type == 'stochastic':
                random_idx = np.random.randint(n_samples)
                X_single = X_with_bias[random_idx:random_idx+1]  # Сохраняем размерность
                y_single = y[random_idx:random_idx+1]
                gradient = self.calc_gradient(X_single, y_single)
                self.w = self.w - self.eta * gradient
            else:
                raise Exception('Unknown gd_type')

            current_loss = self.calc_loss(X_with_bias, y)
            self.loss_history.append(current_loss)
            
            weight_change = np.linalg.norm(self.w - w_prev)
            if weight_change < self.tolerance:
                print(f"Сходимость достигнута на итерации {iteration}")
                break
            w_prev = self.w.copy()
        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        n_samples = X.shape[0]
        X_with_bias = np.c_[np.ones(n_samples), X]
        # Предсказание: y_pred = X * w
        return X_with_bias @ self.w
    
    def calc_gradient(self, X, y):
        """
        X: np.array of shape (ell, d) (ell can be equal to 1 if stochastic)
        y: np.array of shape (ell)
        ---
        output: np.array of shape (d)
        """
        n_samples = X.shape[0]
        y_pred = X @ self.w
        errors = y_pred - y
        # Градиент: (2/n) * X^T * (X*w - y)
        gradient = (2.0 / n_samples) * (X.T @ errors) + 2 * self.alpha * self.w
        return gradient

    def calc_loss(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: float 
        """ 
        n_samples = X.shape[0]
        # Вычисляем предсказания
        y_pred = X @ self.w
        # MSE: (1/n) * sum((y_pred - y)^2)
        mse = np.mean((y_pred - y) ** 2)
        return mse

**Задание 3**. 
* Загрузите данные из лабораторной работы № 3 ([train.csv](https://www.kaggle.com/c/nyc-taxi-trip-duration/data));
* Разбейте выборку на обучающую и тестовую в отношении 7:3 с random_seed=0;
* Преобразуйте целевую переменную `trip_duration` как $\hat{y} = \log{(y + 1)}$.

In [18]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
df = pd.read_csv('train.csv')
numerical_features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 
                      'passenger_count']
X = df[numerical_features]
y = np.log1p(df['trip_duration'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
df.head(1)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.96463,40.765602,N,455


**Задание 4**. Обучите и провалидируйте модели на данных из предыдущего пункта, сравните качество между методами по метрикам MSE и $R^2$. Исследуйте влияние параметров `max_iter` и `eta` на процесс оптимизации. Согласуется ли оно с вашими ожиданиями?

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_error

reg = LinearReg(gd_type='full', tolerance=1e-6, max_iter=1000, eta=1e-6, alpha=15)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

print("=== ОЦЕНКА КАЧЕСТВА МОДЕЛИ ===")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")

Сходимость достигнута на итерации 920
=== ОЦЕНКА КАЧЕСТВА МОДЕЛИ ===
MAE: 0.61
MSE: 0.63
RMSE: 0.79
